In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score
import numpy as np
import time
import warnings

In [2]:
class FullPrecisionLeNet5(nn.Module):
    def __init__(self):
        super(FullPrecisionLeNet5, self).__init__()
        
        # First convolutional layer
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # First convolutional layer
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(x, 2)
        
        # Second convolutional layer
        x = F.relu(self.conv2(x))
        x = F.avg_pool2d(x, 2)
        
        # Flatten the output
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)

In [3]:
def load_data(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
    return train_loader, test_loader

In [4]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [5]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')
    return accuracy

In [6]:
model_name = 'lenet5_fullprecision.pth'

In [7]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if not torch.cuda.is_available():
        print("CUDA is not available. Running on CPU.")
    else:
        print(f"Running on GPU: {torch.cuda.get_device_name(0)}")
    
    model = FullPrecisionLeNet5().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    train_loader, test_loader = load_data()
    
    print(model)
    
    best_accuracy = 0
    for epoch in range(1, 11):
        train(model, device, train_loader, optimizer, criterion, epoch)
        accuracy = test(model, device, test_loader, criterion)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), model_name)
    
    print(f"Best test accuracy: {best_accuracy:.2f}%")
    
    # Load the best model and save in ONNX format
    model = FullPrecisionLeNet5().to(device)
    model.load_state_dict(torch.load(model_name, map_location=device))
    model.eval()
    
    # save_onnx(model, device)

In [10]:
if __name__ == '__main__':
    main()

Running on GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

FullPrecisionLeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301962
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.110744
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.290558
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.221830
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.168652
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.249568
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.087353
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.223373
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.039536
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.057936

Test set: Average loss: 0.0001, Accuracy: 9727/10000 (97.27%)

Train Epoch: 2 [0/60000 (0

In [13]:
def load_model(model_path):
    model = FullPrecisionLeNet5()
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
    model.eval()
    return model

In [14]:
def evaluate_mnist_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    all_probs = []
    correct = 0
    total = 0
    inference_times = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            start_time = time.time()
            output = model(data)
            inference_time = time.time() - start_time
            inference_times.append(inference_time)

            probs = F.softmax(output, dim=1)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    accuracy = 100 * correct / total
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    all_probs = np.array(all_probs)

    # Confusion Matrix
    conf_matrix = confusion_matrix(all_targets, all_preds)

    # Precision, Recall, F1-Score
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)

    # ROC AUC Score (one-vs-all)
    roc_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr', average='weighted')

    # Top-3 Accuracy
    top3_preds = np.argsort(all_probs, axis=1)[:, -3:]
    top3_correct = np.any(top3_preds == all_targets.reshape(-1, 1), axis=1).sum()
    top3_accuracy = 100 * top3_correct / total

    # Inference Time
    avg_inference_time = sum(inference_times) / len(inference_times)

    # Model Size
    model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)

    results = {
        'Accuracy': accuracy,
        'Confusion Matrix': conf_matrix,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC AUC Score': roc_auc,
        'Top-3 Accuracy': top3_accuracy,
        'Average Inference Time (s)': avg_inference_time,
        'Model Size (parameters)': model_size
    }

    return results

# Usage
# Main execution
if __name__ == "__main__":
    model_path = model_name
    batch_size = 64
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the model
    model = load_model(model_path)
    model.to(device)

    # Load the data
    train_loader, test_loader = load_data(batch_size)

    # Evaluate the model
    results = evaluate_mnist_model(model, test_loader, device)
    
    # Print results
    for metric, value in results.items():
        print(f"{metric}:")
        print(value)
        print()

Accuracy:
99.03

Confusion Matrix:
[[ 978    0    0    0    0    1    0    0    0    1]
 [   0 1132    1    0    0    0    2    0    0    0]
 [   1    0 1027    0    0    0    0    3    1    0]
 [   0    0    1 1000    0    3    0    3    3    0]
 [   0    0    1    0  962    0    4    4    1   10]
 [   2    0    0    4    0  883    2    0    0    1]
 [   7    2    1    0    1    5  941    0    1    0]
 [   0    1    2    0    0    0    0 1023    2    0]
 [   2    0    2    0    0    0    2    1  966    1]
 [   1    3    0    2    2    1    0    7    2  991]]

Precision:
0.9903224723825033

Recall:
0.9903

F1-Score:
0.990293975418881

ROC AUC Score:
0.9999301948125896

Top-3 Accuracy:
99.95

Average Inference Time (s):
0.0012232065200805664

Model Size (parameters):
61706

